**Part 1: Spark RDD API** 


Preliminaries

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null &> /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz &> /dev/null
!tar xf spark-3.0.0-bin-hadoop3.2.tgz &> /dev/null
!pip install -q findspark &> /dev/null

In [2]:
import os
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark import SparkContext, SparkFiles
sc = SparkContext.getOrCreate()
from os import path

In [3]:
if os.getcwd().split("/")[-1]=="src":
  %cd ..
dir = os.getcwd()
!mkdir src 
!mkdir out 
%cd ./src 

/content/src


**TASK 1**

Download the data file and make it accessible to Spark

In [4]:
url = "https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/groceries.csv"
spark.sparkContext.addFile(url)

In [5]:
task1rdd=sc.textFile(SparkFiles.get("groceries.csv")).map(lambda x: x.split(","))

In [6]:
task1rdd.take(5)

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'],
 ['tropical fruit', 'yogurt', 'coffee'],
 ['whole milk'],
 ['pip fruit', 'yogurt', 'cream cheese ', 'meat spreads'],
 ['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product']]

**TASK 2**

List of all (unique) products

In [7]:
task2rdd = task1rdd.flatMap(lambda x: x).map(lambda x: "product "+x).distinct()

In [8]:
task2rdd.take(5)

['product citrus fruit',
 'product semi-finished bread',
 'product yogurt',
 'product coffee',
 'product whole milk']

In [18]:
try:
  task2rdd.saveAsTextFile(dir+"/out/out_1_2a.txt")
except:
  print("File already created")

Count elements

In [10]:
task2rddcount =  sc.parallelize(["Count : ",task2rdd.count()])

In [19]:
try:
  task2rddcount.saveAsTextFile(dir+"/out/out_1_2b.txt")
except:
  print("File already created")

**TASK 3**

In [12]:
sc.parallelize(task1rdd.collect()).flatMap(lambda x: x).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], ascending=False).take(5)

[('whole milk', 2513),
 ('other vegetables', 1903),
 ('rolls/buns', 1809),
 ('soda', 1715),
 ('yogurt', 1372)]

In [13]:
task3rdd = sc.parallelize(sc.parallelize(task1rdd.collect()).flatMap(lambda x: x).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], ascending=False).take(5))

In [17]:
try:
  task3rdd.saveAsTextFile(dir+"/out/out_1_3.txt")
except:
  print("File already created")

In [ ]:
from google.colab import files
files.download(dir+"/out")